# YOLO를 이해하자 v1
You Only Look Once: Unified, Real-Time Object Detection https://arxiv.org/pdf/1506.02640.pdf

<img src="img/yologo.png" width="30%">

코드 출처 : https://github.com/pbcquoc/yolo <br>
관련 블로그 : https://pbcquoc.github.io/yolo/

In [1]:
import tensorflow as tf

import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets
from tensorflow.contrib.slim.nets import vgg

from sklearn.model_selection import train_test_split
import json

import numpy as np
import cv2
import matplotlib.pyplot as plt

import time

각 그리드에 대해서 2개씩의 앵커박스를 예측한다.


전체 그리드는 7x7


우리는  (nbox + 4* nbox + nclass) 차원을 가진 벡터를 추측해야 한다.

그래서 2개의 상자, 3개의 클래스인 경우

7x7x13 3차원의 행렬을 만든다.


object1 | object2 | offset x1 | offset y1 | width1 | height2 | offset x1 | offset y1 | width1 | height2 | 클래스1, 클래스2 클래스 3... |

In [2]:
cell_size = 7

box_per_cell = 2
img_size = 224

classes = {'circle': 0, 'triangle':1, 'rectangle': 2}
nclass = len(classes)


box_scale = 5.0
noobject_scale = 0.5
batch_size = 128
epochs = 10

lr = 1e-3

# 데이터 셋 로드 
우리는 크기가 각각 224x224 인 25k 개의 데이터 샘플을 가지고 있으며 json 파일에는 객체가 들어있는 
경계 상자의 배치에 해당하는 객체 세트를 비롯하여 이미지 이미지가 포함되어 있습니다.

데이터를 준비하려면 모든 이미지를 메모리에로드하고 경계 상자의 레벨 정보를 위의 레이블 정보로 바꿔야합니다.

vector를 만들 것인데 우리가 다루기 위한 벡터를 만든다.

In [3]:
def load():
    labels = json.load(open('train/train/labels.json'))
    N = len(labels)
    X = np.zeros((N, img_size, img_size, 3), dtype='uint8')
    y = np.zeros((N, cell_size, cell_size, 5 + nclass))
    for idx, label in enumerate(labels):
        img = cv2.imread("train/train/{}.png".format(idx))
        X[idx] = img
        for box in label['boxes']:
            x1, y1 = box['x1'], box['y1']
            x2, y2 = box['x2'], box['y2']
            # 객체 라벨의 one-hot vector 
            cl = [0]*len(classes)         # 길이 클래스개를 만들고 
            
            cl[classes[box['class']]] = 1 #  
            
            x_center, y_center, w, h = (x1+x2)/2.0, (y1+y2)/2.0, x2-x1, y2-y1
            x_idx, y_idx = int(x_center/img_size*cell_size), int(y_center/img_size*cell_size)
            
            y[idx, y_idx, x_idx]= 1, x_center, y_center, w, h, *cl 
                                     # 배열의 원소를 넣어줄 때 *을 붙인다.
                                     # cl은 ground truth의 onehot이다.
                                     # 해석해 보자면 
    
    return X, y

In [4]:
import wget
url ='https://docs.google.com/uc?export=download&id=1rZufpopTpqjeMNGD1bHgORTw7WF4mmug'
filename = wget.download(url)

-1 / unknown

# train.zip 압축 풀기

In [5]:
X, y = load()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)

각 사각형에 대해 어떤 정보를 예측해야하는지, 다음 요구 사항에 따라 적절한 모양으로 출력을 제공하는 
CNN 모델을 만드는 것이 중요하다. 

즉, gridsize x gridsize x (nbox + 4*nbox +nclass) 

gridsize의 경우, 예를 들어 7x7은 각각 2개의 상자로 예측하여 CNN 모델에서 7x7x13 모양을 출력하는 데 필요한 모든 개체가 3가지 유형이다.

YOLO는 선형 회구를 사용하여 각 사각형의 정보를 예측한다. 
따라서 마지막 레이어에서는 활성화 함수를 전혀 사용하지 않는다. 448x448의 입력 이미지로 CNN 모델은 2x2 크기의 6개의 max pooling을 사용하여 출력 크기에서 이미지의 크기의 64배를 7x7로 줄인다. 동시에 마지막 층의 전체 연결을 바닥을 사용하는 대신 7x7x13의 출력 모양을 쉽게 만들 수 있도록 1x1 conv레이어로 13개의 feature map을 대체한다. 

In [6]:
def vgg16(inputs, is_training):
    '''
    args:
        inputs : 5-D tensor [batch_size, width, height, 3]
    return:
        iou    : 4-D tensor [batch_size, 7 , 7, 5*nbox + nclass]
    '''
    with tf.device("/gpu:0"):
        with tf.variable_scope("vgg_16"):
            with slim.arg_scope(vgg.vgg_arg_scope()):
                net = slim.repeat(inputs, 2, slim.conv2d, 16, [3,3] ,scope='conv1')
                net = slim.max_pool2d(net, [2, 2], scope='pool1')
                net = slim.repeat(net,   2, slim.conv2d, 32, [3,3], scope='conv2')
                net = slim.max_pool2d(net, [2, 2], scope='pool2')
                net = slim.repeat(net,   2, slim.conv2d, 64, [3, 3], scope='conv3')
                net = slim.max_pool2d(net, [2, 2], scope='pool3')
                net = slim.repeat(net,   2, slim.conv2d, 128, [3, 3], scope='conv4')
                net = slim.max_pool2d(net, [2, 2], scope='pool4')
                net = slim.repeat(net,   2, slim.conv2d, 256, [3, 3], scope='conv5')
                net = slim.max_pool2d(net, [2, 2], scope='pool5')
                net = slim.conv2d(net, 512, [1,1], scope='fc6')
                # 마지막층엔 Activation 없다. 
                net = slim.conv2d(net, 13, [1,1], activation_fn=None, scope='fc7')
    return net

IOU 기능

![iou](img/iou.png)

IOU는 모델의 성능 평가를 위한 기준이다. Union 분의 Intersection으로 구한다.

=> intersection / Union

In [7]:
def compute_iou(boxes1, boxes2, scope='iou'):
    """calculate ious
    Args:
      boxes1: 5-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4] 
              마지막 4의 정체 ====> (x_center, y_center, w, h)
              
      boxes2: 5-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4] 
              마지막 4의 정체 ====> (x_center, y_center, w, h)
    Return:
      iou:    4-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    """
    with tf.variable_scope(scope):
        # transform(x_center, y_center, w, h) to (x1, y1, x2, y2)
        # spread 연산 
        
        # center, width, height로 역으로 좌표로 변환
        # spread 연산을 사용한다. 
        boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,  # 이건 벡터다. x1들의 집합이 된다.
                             boxes1[..., 1] - boxes1[..., 3] / 2.0,  # 이건 벡터다. y1들의 집합이 된다.
                             boxes1[..., 0] + boxes1[..., 2] / 2.0,  # 이건 벡터다. x2들의 집합이 된다.
                             boxes1[..., 1] + boxes1[..., 3] / 2.0], # 이건 벡터다. y2들의 집합이 된다.
                            axis=-1)
        
        boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                             boxes2[..., 1] - boxes2[..., 3] / 2.0,
                             boxes2[..., 0] + boxes2[..., 2] / 2.0,
                             boxes2[..., 1] + boxes2[..., 3] / 2.0],
                            axis=-1) 
        
        # calculate the left up point & right down point
        # box1 (x1, y1) 
        # box2 (x2, y2) 와 비교 
        
        # 각 엘리먼트에서 하나씩 뽑아서 노란 사각형의 left_up 좌표 구함 
        left_up    = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
        # 노란 사각형의 right_down 좌표
        right_down = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])
        
        # intersection 노랑색 사각형 넓이 구하기
        intersection = tf.maximum(0.0, right_down-left_up)
        inter_square = intersection[..., 0] * intersection[..., 1]
        
        # calculate the boxes1 square and boxes2 square
        # square1 =>  width*height
        square1 = boxes1[..., 2]*boxes1[..., 3]
        square2 = boxes2[..., 2]*boxes2[..., 3]
        union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
        
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0 )


# Loss function

yolo는 multi loss function을 사용하며 loss function들은 다음과 같다.

* classfication loss 
* localization loss
* confidence loss

훈련 중에 모델은 물체가 포함된 사각형을 볼 것이다. 해당 개체의 올바른 클래스 점수를 올려야 한다.
그런 다음 해당 상자를 보고 예상되는 2개의 상자에서 가장 좋은 경계 상자를 찾는다. 경계 상자의 지역화 점수를 높이고 경계 상자 정보를 레이블과 비슷하게 변경! <br>
물체를 포함하지 않는 경우 실뢰도 낮추고 이 사각형의 점수는 don't care


1. Classification Loss
이 Loss는 예측된 레이블과 올바른 레이블 사이에 제곱된 오차이다.

![closs](img/closs.png)

p'i(c) 모델이 예측하는 해당 사각형에서 클래스 c에 해당하는 확률

# Localization Loss

![lloss](img/lloss.png)


# Confidence Loss
![cfloss](img/cfloss.png)

# Total Loss
![tloss](img/tloss.png)

In [8]:
def loss_layer(predicts, labels, scope='loss_layer'):
    
    """calculate loss function
    Args:
      predicts: 4-D tensor [batch_size, 7, 7, 5*nbox+n_class] 
      labels: 4-D tensor [batch_size, 7, 7, 5+n_class]
    Return:
      loss: scalar
    """
    
    with tf.variable_scope(scope):
        offset = np.transpose( np.reshape(np.array( [np.arange(cell_size)] * cell_size * box_per_cell),
                               (box_per_cell, cell_size, cell_size)),
                               (1, 2, 0 ) )
        
        offset = offset[None, :]
        # (None, cell_size, cell_size, box_per_cell)
        
        offset = tf.constant(offset, dtype= tf.float32)
        offset_tran = tf.transpose(offset, (0,2,1,3))
        
        # 사각형 내 예측 벡터의 첫 번째 요소는 confidence score이다.
        predict_object = predicts[..., :box_per_cell]
        
        # bound box 및 width, height 예측
        predict_box_offset = tf.reshape( predicts[..., box_per_cell:5*box_per_cell],
                                         (-1, cell_size, cell_size, box_per_cell, 4) )
        
        # 마지막 요소는 객체 클래스의 예측이다.
        predict_class = predicts[..., 5*box_per_cell:]
        
        # 오프셋 위치를 (0 ~ 1) 좌표 노말라이즈로 한다.
        predict_normalized_box = tf.stack(
                                [ (predict_box_offset[..., 0] + offset) / cell_size,
                                  (predict_box_offset[..., 1] + offset_tran) / cell_size, 
                                  tf.square(predict_box_offset[..., 2]),
                                  tf.square(predict_box_offset[..., 3]) ], axis=-1)
        
        # 대응하는 라벨을 얻는다. 
        true_object = labels[..., :1]
        true_box = tf.reshape(labels[..., 1:5], (-1, cell_size, cell_size, 1, 4))
        
        # 픽셀 좌표를 단락 (0, 1)로 정규화하려면 img_size 224로 나눈다.
        true_normalized_box = tf.tile(true_box, (1, 1, 1, box_per_cell, 1))/img_size
        true_class = labels[...,5:]
        
        # 레이블에서 offset 위치 계산
        true_box_offset = tf.stack(
                                    [true_normalized_box[..., 0] * cell_size - offset,
                                     true_normalized_box[..., 1] * cell_size - offset_tran,
                                    tf.sqrt(true_normalized_box[..., 2]),
                                    tf.sqrt(true_normalized_box[...,3])],
                            axis=-1)
        
        # iou count
        predict_iou = compute_iou(true_normalized_box, predict_normalized_box)
        
        # 마스크는 정사각형을 포함하는 객체의 위치를 포함
        object_mask = tf.reduce_max(predict_iou, 3, keepdims=True)
        
        # 모니터할 측정 항목
        iou_metric = tf.reduce_mean(tf.reduce_sum(object_mask, axis=[1,2,3])/ tf.reduce_sum(true_object,
                                                                                          axis=[1,2,3]))

        object_mask = tf.cast((predict_iou>=object_mask), tf.float32)* true_object
        noobject_mask = tf.ones_like(object_mask) - object_mask
        
        ## class loss 
        class_delta = true_object*(predict_class - true_class)
        class_loss = tf.reduce_mean(tf.reduce_sum(tf.square(class_delta), axis=[1,2,3]), name='class_loss')
        
        ## object loss
        object_delta = object_mask*(predict_object - predict_iou)
        object_loss = tf.reduce_mean(tf.reduce_sum(tf.square(object_delta), axis=[1,2,3]), name='object_loss')
        
        ## noobject loss
        noobject_delta = noobject_mask*predict_object 
        noobject_loss = tf.reduce_mean(tf.reduce_sum(tf.square(noobject_delta), axis=[1,2,3]), name='noobject_loss')
        
        ## coord loss
        box_mask  = tf.expand_dims(object_mask, 4)
        box_delta = box_mask*(predict_box_offset - true_box_offset)
        box_loss  = tf.reduce_mean(tf.reduce_sum(tf.square(box_delta), axis=[1,2,3]), name='box_loss')
        
        loss = 0.5*class_loss + object_loss + 0.1*noobject_loss + 10*box_loss
        
        return loss, iou_metric, predict_object, predict_class, predict_normalized_box
    

# 그래프 컴파일

In [9]:
graph = tf.Graph()
with graph.as_default():
    images      = tf.placeholder("float", [None, img_size,  img_size,  3], name="input")
    labels      = tf.placeholder("float", [None, cell_size, cell_size, 8], name="label")
    is_training = tf.placeholder(tf.bool)
    
    logits = vgg16(images, is_training)
    loss, iou_metric, predict_object, predict_class, predict_normalized_box = loss_layer(logits, labels)
    
    optimizer = tf.train.AdamOptimizer(lr)
    train_op  = optimizer.minimize(loss)
    

In [10]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # 새미이지를 테스트할 때 모델의 weight를 저장하는 Saver
    saver = tf.train.Saver(max_to_keep=1)
    
    for epoch in range(epochs):
        start_time = time.time()
        for batch in range(len(X_train)//batch_size):
            # adam에 따라 optimization 
            X_batch = X_train[batch*batch_size: (batch+1)*batch_size]
            y_batch = y_train[batch*batch_size: (batch+1)*batch_size]
            train_total_loss, train_iou_m ,_ = sess.run([loss, iou_metric, train_op], 
                                                        {images:X_batch,
                                                         labels:y_batch,
                                                         is_training:True})
        end_time = time.time()
        
        # 이부분은 valiation loss를 확인
        val_loss   = []
        val_iou_ms = []
        for batch in range(len(X_test)//batch_size):
            val_X_batch = X_test[ batch*batch_size : (batch+1)*batch_size]
            val_y_batch = y_test[ batch*batch_size : (batch+1)*batch_size]
            total_val_loss, val_iou_m, val_predict_object, val_predict_class, val_predict_normalized_box = sess.run([loss, iou_metric, predict_object, predict_class, predict_normalized_box], 
                                                 {images:val_X_batch, labels:val_y_batch, is_training:False})
            
            val_loss.append(total_val_loss)
            val_iou_ms.append(val_iou_m)
            
        
        saver.save(sess, './model/yolo', global_step=epoch)
        print('epoch: {} - running_time: {:.0f}s - train_loss: {:.3f} - train_iou: {:.3f} - val_loss: {:.3f} - val_iou: {:.3f}'.format(epoch, end_time - start_time, train_total_loss, train_iou_m, np.mean(val_loss), np.mean(val_iou_ms)))

InvalidArgumentError: Cannot assign a device for operation 'vgg_16/conv1/conv1_1/weights/Initializer/random_uniform/RandomUniform': Could not satisfy explicit device specification '' because the node was colocated with a group of nodes that required incompatible device '/device:GPU:0'
Colocation Debug Info:
Colocation group had the following types and devices: 
L2Loss: CPU 
Assign: GPU CPU 
Add: GPU CPU 
Const: GPU CPU 
RandomUniform: GPU CPU 
Sub: GPU CPU 
VariableV2: GPU CPU 
Mul: GPU CPU 
Identity: GPU CPU 
ApplyAdam: GPU CPU 

Colocation members and user-requested devices:
  vgg_16/conv1/conv1_1/weights/Initializer/random_uniform/shape (Const) 
  vgg_16/conv1/conv1_1/weights/Initializer/random_uniform/min (Const) 
  vgg_16/conv1/conv1_1/weights/Initializer/random_uniform/max (Const) 
  vgg_16/conv1/conv1_1/weights/Initializer/random_uniform/RandomUniform (RandomUniform) 
  vgg_16/conv1/conv1_1/weights/Initializer/random_uniform/sub (Sub) 
  vgg_16/conv1/conv1_1/weights/Initializer/

# Prediction 결과 표시

조건에 맞지 않는 모든 상자 필터링, 개체 포함 안함 <br>
Non Maximum supression - 여러개의 겹침 상자 병합

In [ ]:
def iou(box1, box2):
    """ tính iou bằng numpy 
    Args:
      box1: [center_x, center_y, w, h] 
      box2: [center_x, center_y, w, h] 
    Return:
      iou: iou
    """
    tb = min(box1[0] + 0.5 * box1[2], box2[0] + 0.5 * box2[2]) - \
        max(box1[0] - 0.5 * box1[2], box2[0] - 0.5 * box2[2])
    lr = min(box1[1] + 0.5 * box1[3], box2[1] + 0.5 * box2[3]) - \
        max(box1[1] - 0.5 * box1[3], box2[1] - 0.5 * box2[3])
    inter = 0 if tb < 0 or lr < 0 else tb * lr
    return inter / (box1[2] * box1[3] + box2[2] * box2[3] - inter)
    
def interpret_output(predict_object, predict_class, predict_normalized_box):
    # 노말라이즈 된거에 인풋 사이즈 곱
    predict_box= predict_normalized_box*img_size
    predict_object = np.expand_dims(predict_object, axis=-1)
    predict_class = np.expand_dims(predict_class, axis=-2)
    
    #
    class_probs = predict_object*predict_class
    
    # 레이어를 포함할 경계 상자 유지 >= 0.2
    filter_mat_probs = np.array(class_probs >= 0.2, dtype='bool')
    filter_mat_boxes = np.nonzero(filter_mat_probs)
    boxes_filtered = predict_box[filter_mat_boxes[0], filter_mat_boxes[1], filter_mat_boxes[2]]
    class_probs_filtered = class_probs[filter_mat_probs]
    
    # 클래스 인덱스를 선택해서 각 바운드 박스의 경계 레이어 결정 
    classes_num_filtered = np.argmax(
        filter_mat_probs, axis=3)[
        filter_mat_boxes[0], filter_mat_boxes[1], filter_mat_boxes[2]]

    # 가능성이 가장 높은 확률을 예측하는 바운드 박스로 남김
    argsort = np.array(np.argsort(class_probs_filtered))[::-1]
    boxes_filtered = boxes_filtered[argsort]
    class_probs_filtered = class_probs_filtered[argsort]
    classes_num_filtered = classes_num_filtered[argsort]

    #  non-maximun suppression 계산
    for i in range(len(boxes_filtered)):
        if class_probs_filtered[i] == 0:
            continue
        for j in range(i + 1, len(boxes_filtered)):
            if iou(boxes_filtered[i], boxes_filtered[j]) > 0.5:
                class_probs_filtered[j] = 0.0
                
    # 마지막 단계의 필터로 논맥시점으로 오버랩 제거
    filter_iou = np.array(class_probs_filtered > 0.0, dtype='bool')
    boxes_filtered = boxes_filtered[filter_iou]
    class_probs_filtered = class_probs_filtered[filter_iou]
    classes_num_filtered = classes_num_filtered[filter_iou]

    result = []
    for i in range(len(boxes_filtered)):
        result.append(
            [classes_num_filtered[i],
             boxes_filtered[i][0],
             boxes_filtered[i][1],
             boxes_filtered[i][2],
             boxes_filtered[i][3],
             class_probs_filtered[i]])

    return result

def draw_result(img, result):
    """ 예상되는 결과 표시
    Args:
      img: 이미지
      result: 생성된 이미지 값    
    """
    plt.figure(figsize=(10,10), dpi=40)
    img = np.pad(img, [(50,50), (50,50), (0,0)], mode='constant', constant_values=255)
    for i in range(len(result)):
        x = int(result[i][1])+50
        y = int(result[i][2])+50
        w = int(result[i][3] / 2)
        h = int(result[i][4] / 2)
        cv2.rectangle(img, (x - w, y - h), (x + w, y + h), (231, 76, 60), 2)
        cv2.rectangle(img, (x - w, y - h - 20),
                      (x -w + 50, y - h), (46, 204, 113), -1)
        cv2.putText(
            img, '{} : {:.2f}'.format(result[i][0] ,result[i][5]),
            (x - w + 5, y - h - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.3,
            (0, 0, 0), 1, cv2.LINE_AA)

    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

In [ ]:
img_idx = 15
result = interpret_output(val_predict_object[img_idx], val_predict_class[img_idx], val_predict_normalized_box[img_idx])

In [ ]:
draw_result(val_X_batch[img_idx]*255, result)